## Definition of the Problem 
The main target of this data is to model the Customer Intelligence in the Bank. Banks in general needs to analyse the banks client intelligence (behaviour) and the estimation of clients behaviour  with inclusion of the external factors could be very useful for the Bank management to predict if the client is active or non-active. If the client is non-active there is an indication to take pro-active action to the client to keep him as a stable Bank client for the future and do not let him switch the bank. 
The Customer intelligence is described by 36 dimensional data (6 are nominal and 30 are real). The associations of the data with particular customer as well as the real meaning of each of presented 36 features are matter of confidentiality and will not be revealed. 
The Bank is providing 24 000 representation data for a competition. 12 000 are marked as "Active" and 12 000 are marked as "non-active" clients. 

## The goal 
To create the classification tools for distinguishing the "active" and "non-active" classes - dichotomise classification. 

## Description of training and testing files 
client_train.txt	- the file consists of 12 000 training patterns. Each pattern has 37 values (**36 are input attributes**, 37th is output class), **first 6 represent values of nominal attributes** following 30 represent values of real attributes and the last value represents information about class. 

client_test.txt	- the file consists of 12 000 testing patterns. Each pattern has the same structure as patterns in client_train.txt file without information about class. 


### 1. Perform a exploratory data analysis and discuss what you have learned from this analysis.

Loading the data:

In [ ]:
options(warn = FALSE)
train_data <- try(read.table("Input/client_train.txt"))
if (class(train_data) == "try-error") {
    folder = "Input/"
    file = "client_train.txt"
    cat("Downloading the data instead.")
    dir.create(folder)
    train_data <- read.table("http://web.tecnico.ulisboa.pt/~ist13493/MEDM2020/Project1/Group3Data/client_train.txt")
    write.table(train_data, file=paste(folder, file, sep=''), append = FALSE, sep = " ", dec = ".",
            row.names = FALSE, col.names = FALSE)
    }
test_data <- try(read.table("Input/client_test_marked.txt"))
if (class(train_data) == "try-error") {
    file = "client_test_marked.txt"
    print("Downloading the data instead.")
    train_data <- read.table("http://web.tecnico.ulisboa.pt/~ist13493/MEDM2020/Project1/Group3Data/client_test_marked.txt")
    write.table(train_data, file=paste(folder, file, sep=''), append = FALSE, sep = " ", dec = ".",
            row.names = FALSE, col.names = FALSE)
    }
options(warn = TRUE)

Loading the required packages:

In [ ]:
library(ggplot2)
library(e1071)
library(DataExplorer)

We begin by checking the data:

In [ ]:
head(train_data)

We present a summary of descriptive statistics for the numeric variables:

In [ ]:
v <-sapply(train_data[7:36], var)
m <-sapply(train_data[7:36], mean)
q <-sapply(train_data[7:36], quantile)
i <-sapply(train_data[7:36], IQR)
s <-sapply(train_data[7:36], skewness)
k <-sapply(train_data[7:36], kurtosis)

descriptive <- data.frame("mean" = m, "variance" = v, "minimum" = q[1,], "Q1" = q[2,], "median" = q[2,], 
                          "Q3" = q[3,], "max" = q[4,], "IQR" = i, "skewness" = s, "kurtosis" = k)

show = TRUE
if(show){
    descriptive
} else {
    folder = "Output/"
    file = "descriptive.csv"
    suppressWarnings(dir.create(folder))
    write.csv(descriptive, file = paste(path, file, sep = ''))
}

We can observe that...


Next we analyse the correlation of the numeric values and also the label:

In [ ]:
plot_correlation(train_data[, 7:37])

In [ ]:
#qplot(log(log(train_data$V12+1)), log(log(train_data$V14+1)), colour = train_data$V37)

In [ ]:
#sp <- ggplot(train_data, aes(log(log(train_data$V12+1)), log(log(train_data$V14+1)+1), colour = train_data$V37))
#sp + geom_point(alpha=.15)

In [ ]:
#sp <- ggplot(train_data, aes(log(log(train_data$V12+1)+1), log(log(train_data$V14+1)+1)))
#sp + geom_point() + stat_density2d()

In [ ]:
#addmargins(table(train_data$V3, train_data$V37))

In [ ]:
class1 <- data.frame(V34 = train_data[train_data[37]=='_1_', 34])
class0 <- data.frame(V34 = train_data[train_data[37]=='_0_', 34])

class1$Class <- 'Class 1'
class0$Class <- 'Class 0'

classCounts <- rbind(class1, class0)

ggplot(classCounts, aes(V34, fill = Class)) + 
   geom_histogram(alpha = 0.5, aes(y = ..count..), position = 'identity', bins = 100)